In [1]:
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import time
import os
import copy
import math
from utilities import *

print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
print("GPU is available?", torch.cuda.is_available())

PyTorch Version: 1.11.0+cu113
Torchvision Version: 0.12.0+cu113
GPU is available? True


In [2]:
dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Imported datasets
For the testing and comparison of our algorithms we will use the following datasets:

1. MNIST
2. FashionMNIST
3. CIFAR10

In [3]:
ts = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0,), (1,))])
mnist_trainset = datasets.MNIST('../data', train=True, download=True, transform=ts)
mnist_testset = datasets.MNIST(root='../data', train=False, download=True, transform=ts)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [4]:
fmnist_trainset = datasets.FashionMNIST('../data', train=True, download=True, transform=ts)
fmnist_testset = datasets.FashionMNIST(root='../data', train=False, download=True, transform=ts)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ../data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ../data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ../data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ../data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw



In [5]:
"""
cifar10_trainset = datasets.CIFAR10('../data', train=True, download=True, transform=ts)
cifar10_testset = datasets.CIFAR10(root='../data', train=False, download=True, transform=ts)
"""

"\ncifar10_trainset = datasets.CIFAR10('../data', train=True, download=True, transform=ts)\ncifar10_testset = datasets.CIFAR10(root='../data', train=False, download=True, transform=ts)\n"

In [6]:
"""
from sklearn import datasets
from sklearn.model_selection import train_test_split

boston = datasets.load_boston()
X = boston.data
y = boston.target
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

x_train = torch.from_numpy(x_train).to(device = device)
x_test = torch.from_numpy(x_test).to(device = device)
y_train = torch.from_numpy(y_train).to(device = device)
y_test = torch.from_numpy(y_test).to(device = device)

x_train = x_train.T.float()
x_test = x_test.T.float()

N = x_train.shape[1]
N_test = x_test.shape[1]
"""

'\nfrom sklearn import datasets\nfrom sklearn.model_selection import train_test_split\n\nboston = datasets.load_boston()\nX = boston.data\ny = boston.target\nx_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)\n\nx_train = torch.from_numpy(x_train).to(device = device)\nx_test = torch.from_numpy(x_test).to(device = device)\ny_train = torch.from_numpy(y_train).to(device = device)\ny_test = torch.from_numpy(y_test).to(device = device)\n\nx_train = x_train.T.float()\nx_test = x_test.T.float()\n\nN = x_train.shape[1]\nN_test = x_test.shape[1]\n'

# Train - test split

Code taken from https://github.com/timlautk/BCD-for-DNNs-PyTorch/blob/master/bcd_dnn_mlp_mnist.ipynb

In [7]:
x_train, y_train, x_test, y_test, y_train_one_hot, y_test_one_hot= load_dataset(fmnist_trainset, fmnist_testset, 10)

# We move to GPU
x_train = x_train.to(device = device)
x_test = x_test.to(device = device)
y_train = y_train.to(device = device)
y_test = y_test.to(device = device)
y_train_one_hot = y_train_one_hot.to(device = device)
y_test_one_hot = y_test_one_hot.to(device = device)

# Architecture initialization

For the MultiLayerPerceptron we have the parameters **input_size** , **hidden_size**,**output_size** corresponding to the size of the input layer, the hidden layer and the output layer, respectively.

The MLP only has 3 layers like https://github.com/timlautk/BCD-for-DNNs-PyTorch/blob/master/bcd_dnn_mlp_mnist.ipynb as a starting point.

Also we use ReLU currently for the same reason.

In [8]:
input_size = x_train.shape[0]
hidden_size = 1500
output_size = 10

# Training

Note: Fix it so that it moves everything to device in the following function and that it does the label sample split here

In [9]:
def update_v_js(U1, U2, W, b, rho, gamma):
    """
    The function updates the V_js parameters during the training phase
    
    :param U1: The U parameter on the same level of V that we are updating
    :param U2: The U parameter which is in the next level of the V that we are updating
    :param W: The W parameter which is in the next level of the V that we are updating
    :param b: The b parameter which is in the next level of the V that we are updating
    :param rho: The constant rho parameter which is in the next level of the V that we are updating
    :param gamma: The constant gamma parameter which is in the next level of the V that we are updating
    :return: The updated V
    """
    _, d = W.size()
    I = torch.eye(d, device=device)
    U1 = nn.ReLU()(U1)
    _, col_U2 = U2.size()
    Vstar = torch.mm(torch.inverse(rho * (torch.mm(torch.t(W), W)) + gamma * I),
                     rho * torch.mm(torch.t(W), U2 - b.repeat(1, col_U2)) + gamma * U1)
    return Vstar

In [10]:
def update_wb_js(U, V, W, b, alpha, rho):
    """
    The function updates the W and b parameters during the training phase
    
    :param U: The U in the current level of W and b
    :param V: The V in the previous level with respect to the W that we are updating
    :param W: The current W that we have to update
    :param b: The current b that we have to update
    :param alpha: The alpha constant of the updates
    :param rho: The rho constant of the updates
    :return:
    """
    d, N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha * W + rho * torch.mm(U - b.repeat(1, col_U), torch.t(V)),
                     torch.inverse(alpha * I + rho * (torch.mm(V, torch.t(V)))))
    bstar = (alpha * b + rho * torch.sum(U - torch.mm(W, V), dim=1).reshape(b.size())) / (rho * N + alpha)
    return Wstar, bstar

In [11]:
def relu_prox(a, b, gamma, d, N):
    """
    The function compute the solution to the relu proximal update problem
    
    :param a: the a in the closed formula of the linearized update
    :param b: the b in the closed formula of the linearized update
    :param gamma: The constant used in the update
    :param d: the dimension of the current layer
    :param N: The number of samples
    :return: The obtained solution of the prox update
    """
    val = torch.empty(d, N, device=device)
    x = (a + gamma * b) / (1 + gamma)
    y = torch.min(b, torch.zeros(d, N, device=device))
    # torch.zeros(d,N, device=device)
    val = torch.where(a + gamma * b < 0, y, torch.zeros(d, N, device=device))
    val = torch.where(
        ((a + gamma * b >= 0) & (b >= 0)) | ((a * (gamma - np.sqrt(gamma * (gamma + 1))) <= gamma * b) & (b < 0)), x,
        val)
    val = torch.where((-a <= gamma * b) & (gamma * b <= a * (gamma - np.sqrt(gamma * (gamma + 1)))), b, val)
    return val


In [12]:
def flatten(t):
  """
  The function flatten the list removing the sublists in it
  :param t: the list to flatten
  """
  return [item for sublist in t for item in sublist]


def random_select(min,max,nodes,dims):
  """
  This function generates a set of indexes to chose which nodes of the layer we will update
  :param min: the minimum index from which the nodes will be chosen
  :param max: the maximum index from which the nodes will be chosen
  :param nodes: the number of nodes that should be chosen
  :param dims: the second dimension of the layer 
  :return layers,res: two lists that index together randomly selected elements in the layers
  """
  res = []
  layers = []
  for i in range(dims):
    res.append(random.sample(range(min,max),nodes))
    layers.append([i]*nodes)
  return flatten(layers),flatten(res)

def rand(min,max,nodes):
   return random.sample(range(min,max),nodes)

In [13]:
def index_lists(l1,l2):
  ret1 = flatten(list(map(lambda x : np.repeat(x,len(l2)).tolist(),l1)))
  ret2 = np.tile(l2,len(l1)).tolist()
  return ret1,ret2

In [14]:
def make_pred(Ws,bs,input,N):
  """
  The function is used to make the predictions based on the best found parameters
  :param Ws: The weight matrices
  :param bs: the bias vectors
  :return pred: the predictions
  """
  a1_train = input
  for i in range(0,len(Ws)-1):
    a1_train = nn.ReLU()(torch.addmm(bs[i].repeat(1, N), Ws[i], a1_train))
  pred = torch.argmax(torch.addmm(bs[len(Ws)-1].repeat(1, N), Ws[len(Ws)-1], a1_train), dim=0)
  return pred

In [15]:
#The function requires at least 1 hidden layer otherwise it need some rewrriting
def execute_training(layers, input_size, hidden_size, output_size, train_set, val_set,train_labels, val_labels, niter = 100,
                     gamma = 1, alpha = 5):
  """
  The function takes the following arguements and produces a list of weights and biases with which 
  you can use the make_pred function to get a list of predictions
  :param layers: The total number of layers of the network
  :param input_size: The total size of the input layer
  :param hidden_size: The size of the hidden layer
  :param output_size: The size of the output layer (usefull for multiclass classification)
  :param train_set: The training set
  :param val_set: The validation set
  :param train_labels: The training labels
  :param test labels: The validation labels
  :param niter: The default number of epochs to train the network
  :param gamma: The gamma parameter of the algorithm
  :param alpha: The alpha parameter of the algorithm
  :return Ws,bs: Returns two lists that go in order from the input to the output layer of the weights and the biases of each layer
  """

  N = len(train_labels)
  N_test = len(val_labels)

  # weight initialization (we replicate pytorch weight initialization)

  std = math.sqrt(1/input_size)
  W = torch.FloatTensor(hidden_size, input_size).uniform_(-std, std)
  b = torch.FloatTensor(hidden_size, 1).uniform_(-std, std)

  b = b.to(device = device)
  W = W.to(device = device)

  U = torch.addmm(b.repeat(1, N), W, x_train) # equivalent to W1@x_train+b1.repeat(1,N)
  V = nn.ReLU()(U)

  Ws = [W]
  bs = [b]
  Us = [U]
  Vs = [V]

  for i in range(1,layers-1):
    std = math.sqrt(1/hidden_size)
    W = torch.FloatTensor(hidden_size, hidden_size).uniform_(-std, std)
    b = torch.FloatTensor(hidden_size, 1).uniform_(-std, std)
    b = b.to(device = device)
    W = W.to(device = device)
    U = torch.addmm(b.repeat(1, N), W, Vs[-1])
    V = nn.ReLU()(U)
    Ws.append(W)
    bs.append(b)
    Us.append(U)
    Vs.append(V)
  
  std = math.sqrt(1/hidden_size)
  W = torch.FloatTensor(output_size, hidden_size).uniform_(-std, std)
  b = torch.FloatTensor(output_size, 1).uniform_(-std, std)

  # we move them to GPU
  b = b.to(device = device)
  W = W.to(device = device)
  U = torch.addmm(b.repeat(1, N), W, Vs[-1])
  V = U
  Ws.append(W)
  bs.append(b)
  Us.append(U)
  Vs.append(V)
  
  # constant initialization

  gamma1 = gamma2 = gamma3 = gamma4 = gamma

  rho = gamma
  rho1 = rho2 = rho3 = rho4 = rho

  alpha1 = alpha2 = alpha3 = alpha4 = alpha5 = alpha6 = alpha7 \
  = alpha8 = alpha9 = alpha10 = alpha

  # vector of performance initialization

  loss1 = np.empty(niter)
  loss2 = np.empty(niter)
  accuracy_train = np.empty(niter)
  accuracy_test = np.empty(niter)
  time1 = np.empty(niter)

  print('Train on', N, 'samples, validate on', N_test, 'samples')
  for k in range(niter):

    start = time.time()

    # update V3
    Vs[-1] = (y_train_one_hot + gamma3*Us[-1] + alpha1*Vs[-1])/(1+ gamma3 + alpha1)
    #Vs[-1] = (train_labels + gamma3*Us[-1] + alpha1*Vs[-1])/(1+ gamma3 + alpha1)

    # update U3 
    Us[-1] = (gamma3*Vs[-1] + rho3*(torch.mm(Ws[-1],Vs[-2]) + bs[-1].repeat(1,N)))/(gamma3 + rho3)

    # update W3 and b3
    W, b = update_wb_js(Us[-1],Vs[-2],Ws[-1],bs[-1],alpha1, rho3)
    Ws[-1] = W
    bs[-1] = b

    for i in range(len(Vs)-2,0,-1):
      Vs[i] = update_v_js(Us[i],Us[i+1],Ws[i+1],bs[i+1],rho3,gamma2)
      Us[i] = relu_prox(Vs[i],(rho2*torch.addmm(bs[i].repeat(1,N), Ws[i], Vs[i-1]) +
                               alpha2*Us[i])/(rho2 + alpha2),(rho2 + alpha2)/gamma2, hidden_size, N)
      W,b = update_wb_js(Us[i],Vs[i-1],Ws[i],bs[i],alpha3,rho2)
      Ws[i] = W
      bs[i]= b
    
    # update V1
    Vs[0] = update_v_js(Us[0],Us[1],Ws[1],bs[1],rho2,gamma1)
        
    # update U1
    Us[0] = relu_prox(Vs[0],(rho1*torch.addmm(bs[0].repeat(1,N), Ws[0], x_train) +
                             alpha7*Us[0])/(rho1 + alpha7),(rho1 + alpha7)/gamma1, hidden_size, N)
    
    # update W1 and b1
    W, b = update_wb_js(Us[0],x_train,Ws[0],bs[0],alpha8,rho1)
    Ws[0] = W
    bs[0] = b

    #a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
    #a1_train = x_train
    #for i in range(len(Vs)-1,0,-1):
    #  a1_train = nn.ReLU()(torch.addmm(bs[i].repeat(1, N), Ws[i], a1_train))
    #pred = torch.argmax(torch.addmm(bs[0].repeat(1, N), Ws[0], a1_train), dim=0)
    pred = make_pred(Ws,bs,x_train,N)

    #a1_test = x_test
    #a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
    #for i in range(len(Vs)-1,0,-1):
    #  a1_test = nn.ReLU()(torch.addmm(bs[i].repeat(1, N_test), Ws[i], a1_test))
    #pred_test = torch.argmax(torch.addmm(bs[0].repeat(1, N_test), Ws[0], a1_test), dim=0)
    pred_test = make_pred(Ws,bs,x_test,N_test) 

    loss1[k] = gamma/2*torch.pow(torch.dist(Vs[-1],y_train_one_hot,2),2).cpu().numpy()
    loss2[k] = loss1[k] + gamma/2 * torch.pow(torch.dist(torch.addmm(bs[0].repeat(1,N), Ws[0], x_train),Us[0],2),2).cpu().numpy()

    for i in range(1,layers):
      loss2[k] = loss2[k] + gamma/2 * torch.pow(torch.dist(torch.addmm(bs[i].repeat(1,N), Ws[i], Vs[i-1]),Us[i],2),2).cpu().numpy()

    #loss2[k] = loss1[k] + rho1/2*torch.pow(torch.dist(torch.addmm(b1.repeat(1,N), W1, x_train),U1,2),2).cpu().numpy() \
    #+rho2/2*torch.pow(torch.dist(torch.addmm(b2.repeat(1,N), W2, V1),U2,2),2).cpu().numpy() \
    #+rho3/2*torch.pow(torch.dist(torch.addmm(b3.repeat(1,N), W3, V2),U3,2),2).cpu().numpy()
        
    # compute training accuracy
    correct_train = pred == train_labels
    accuracy_train[k] = np.mean(correct_train.cpu().numpy())
        
    # compute validation accuracy
    correct_test = pred_test == val_labels
    accuracy_test[k] = np.mean(correct_test.cpu().numpy())
        
    # compute training time
    stop = time.time()
    duration = stop - start
    time1[k] = duration
        
    # print results
    print('Epoch', k + 1, '/', niter, '\n', 
          '-', 'time:', time1[k], '-', 'sq_loss:', loss1[k], '-', 'tot_loss:', loss2[k], 
          '-', 'acc:', accuracy_train[k], '-', 'val_acc:', accuracy_test[k])
  print('The total time spent is:', np.sum(time1), 's')
  return Ws, bs

In [16]:
_ , _ = execute_training(3, input_size, hidden_size, output_size, x_train, x_test, y_train, y_test, niter = 25, gamma = 1, alpha = 5)

Train on 60000 samples, validate on 10000 samples
Epoch 1 / 25 
 - time: 1.3042171001434326 - sq_loss: 22858.88671875 - tot_loss: 22891.867291796952 - acc: 0.43505 - val_acc: 0.4288
Epoch 2 / 25 
 - time: 1.298654556274414 - sq_loss: 17264.033203125 - tot_loss: 17295.902361603454 - acc: 0.78675 - val_acc: 0.7718
Epoch 3 / 25 
 - time: 1.3016672134399414 - sq_loss: 13226.94921875 - tot_loss: 13246.962315445766 - acc: 0.8502833333333333 - val_acc: 0.8309
Epoch 4 / 25 
 - time: 1.3144512176513672 - sq_loss: 10196.1728515625 - tot_loss: 10210.698124896735 - acc: 0.8660333333333333 - val_acc: 0.8455
Epoch 5 / 25 
 - time: 1.3332951068878174 - sq_loss: 7877.8447265625 - tot_loss: 7889.606394108385 - acc: 0.87095 - val_acc: 0.85
Epoch 6 / 25 
 - time: 1.321225643157959 - sq_loss: 6091.998046875 - tot_loss: 6102.557561732829 - acc: 0.87345 - val_acc: 0.8523
Epoch 7 / 25 
 - time: 1.3422613143920898 - sq_loss: 4712.55908203125 - tot_loss: 4722.577702738345 - acc: 0.8751166666666667 - val_acc: 0

### Dropout

In [49]:
torch.manual_seed(32)
d0 = input_size
d1 = d2 = hidden_size
d3 = output_size 
N = len(y_train)
N_test = len(y_test)
# The layers are: input + 2 hidden + output

# we represent the weigths of each layer as matrices with d_{i-1} columns and d_{i} rows

# Weight initialization (we replicate pytorch initialization)
std_1 = math.sqrt(1/d0)
W1 = torch.FloatTensor(d1, d0).uniform_(-std_1, std_1)
b1 = torch.FloatTensor(d1, 1).uniform_(-std_1, std_1)

# we move them to GPU
b1 = b1.to('cuda:0')
W1 = W1.to('cuda:0')


std_2 = math.sqrt(1/d1)
W2 = torch.FloatTensor(d2, d1).uniform_(-std_2, std_2)
b2 = torch.FloatTensor(d2, 1).uniform_(-std_2, std_2)

# we move them to GPU
b2 = b2.to('cuda:0')
W2 = W2.to('cuda:0')

std_3 = math.sqrt(1/d2)
W3 = torch.FloatTensor(d3, d2).uniform_(-std_3, std_3)
b3 = torch.FloatTensor(d3, 1).uniform_(-std_3, std_3)

# we move them to GPU
b3 = b3.to('cuda:0')
W3 = W3.to('cuda:0')


U1 = torch.addmm(b1.repeat(1, N), W1, x_train) # equivalent to W1@x_train+b1.repeat(1,N)
V1 = nn.ReLU()(U1)
U2 = torch.addmm(b2.repeat(1, N), W2, V1)
V2 = nn.ReLU()(U2)
U3 = torch.addmm(b3.repeat(1, N), W3, V2)
V3 = U3

# constant initializations
gamma = 1
gamma1 = gamma2 = gamma3 = gamma4 = gamma

rho = gamma
rho1 = rho2 = rho3 = rho4 = rho


alpha = 5
alpha1 = alpha2 = alpha3 = alpha4 = alpha5 = alpha6 = alpha7 \
= alpha8 = alpha9 = alpha10 = alpha

# initialization of the vectors of losses and accuracies
niter = 10
loss1 = np.empty(niter)
loss2 = np.empty(niter)
accuracy_train = np.empty(niter)
accuracy_test = np.empty(niter)
time1 = np.empty(niter)

In [50]:
# we perform one-hot encoding on the y-> y_train_one_hot will be a tensor of dimension 10*training_samples, for each sample there
# will be a one in the row corresponding to the class.

# Iterations
print('Train on', N, 'samples, validate on', N_test, 'samples')
for k in range(niter):

  start = time.time()

  #select_the nodes for the third layer
  #if(niter % 20 == 0):
  nodes_up3 = rand(0,10,10)
  nodes_up2 = rand(0,1500,1000)
  nodes_up1 = rand(0,1500,1000)
  V3_sample = V3[nodes_up3]
  U3_sample = U3[nodes_up3]
  W3_sample = torch.transpose(torch.transpose(W3[nodes_up3],0,1)[nodes_up2],0,1)
  b3_sample = b3[nodes_up3]
  V2_sample = V2[nodes_up2]
  U2_sample = U2[nodes_up2]
  W2_sample = torch.transpose(torch.transpose(W2[nodes_up2],0,1)[nodes_up1],0,1)
  b2_sample = b2[nodes_up2]
  V1_sample = V1[nodes_up1]
  U1_sample = U1[nodes_up1]
  W1_sample = W1[nodes_up1]
  b1_sample = b1[nodes_up1]
  # update V3
  V3_sample = (y_train_one_hot[nodes_up3] + gamma3*U3_sample + alpha1*V3_sample)/(1+ gamma3 + alpha1)
      
  # update U3 
  U3_sample = (gamma3*V3_sample + rho3*(torch.mm(W3_sample,V2_sample) + b3_sample.repeat(1,N)))/(gamma3 + rho3)

  # update W3 and b3
  W3_sample, b3_sample = update_wb_js(U3_sample,V2_sample,W3_sample,b3_sample,alpha1,rho3)
  
  # update V2
  V2_sample = update_v_js(U2_sample,U3_sample,W3_sample,b3_sample,rho3,gamma2)
      
  # update U2
  U2_sample = relu_prox(V2_sample,(rho2*torch.addmm(b2_sample.repeat(1,N), W2_sample, V1_sample) + alpha2*U2_sample)/(rho2 + alpha2),(rho2 + alpha2)/gamma2,1000,N)
      
  # update W2 and b2
  W2_sample, b2_sample = update_wb_js(U2_sample,V1_sample,W2_sample,b2_sample,alpha3,rho2)
  
  # update V1
  V1_sample = update_v_js(U1_sample,U2_sample,W2_sample,b2_sample,rho2,gamma1)
      
  # update U1
  U1_sample = relu_prox(V1_sample,(rho1*torch.addmm(b1_sample.repeat(1,N), W1_sample, x_train) + alpha7*U1_sample)/(rho1 + alpha7),(rho1 + alpha7)/gamma1,1000,N)
  
  # update W1 and b1
  W1_sample, b1_sample = update_wb_js(U1_sample,x_train,W1_sample,b1_sample,alpha8,rho1)

  #print(list(zip(nodes_up3,nodes_up2)))
  #print(W3[].shape)
  V3[nodes_up3] = V3_sample
  U3[nodes_up3] = U3_sample
  W3_sample = W3_sample
  W3[index_lists(nodes_up3,nodes_up2)] = torch.reshape(W3_sample,(1000*10,))
  b3_sample = b3_sample
  b3[nodes_up3] = b3_sample
  V2[nodes_up2] = V2_sample
  U2[nodes_up2] = U2_sample
  W2_sample = W2_sample
  W2[index_lists(nodes_up2,nodes_up1)] = torch.reshape(W2_sample,(1000*1000,))
  b2_sample = b2_sample
  b2[nodes_up2] = b2_sample
  V1[nodes_up1] = V1_sample
  U1[nodes_up1] = V1_sample
  W1_sample = W1_sample
  W1[nodes_up1] = W1_sample
  b1_sample = b1_sample
  b1[nodes_up1] = b1_sample

  a1_train = nn.ReLU()(torch.addmm(b1_sample.repeat(1, N), W1_sample, x_train))
  a2_train = nn.ReLU()(torch.addmm(b2_sample.repeat(1, N), W2_sample, a1_train))
  pred = torch.argmax(torch.addmm(b3_sample.repeat(1, N), W3_sample, a2_train), dim=0)

  a1_test = nn.ReLU()(torch.addmm(b1_sample.repeat(1, N_test), W1_sample, x_test))
  a2_test = nn.ReLU()(torch.addmm(b2_sample.repeat(1, N_test), W2_sample, a1_test))
  pred_test = torch.argmax(torch.addmm(b3_sample.repeat(1, N_test), W3_sample, a2_test), dim=0)
      
  loss1[k] = gamma3/2*torch.pow(torch.dist(V3,y_train_one_hot,2),2).cpu().numpy()
  loss2[k] = loss1[k] + rho1/2*torch.pow(torch.dist(torch.addmm(b1.repeat(1,N), W1, x_train),U1,2),2).cpu().numpy() \
  +rho2/2*torch.pow(torch.dist(torch.addmm(b2.repeat(1,N), W2, V1),U2,2),2).cpu().numpy() \
  +rho3/2*torch.pow(torch.dist(torch.addmm(b3.repeat(1,N), W3, V2),U3,2),2).cpu().numpy()
      
  # compute training accuracy
  correct_train = pred == y_train
  accuracy_train[k] = np.mean(correct_train.cpu().numpy())
      
  # compute validation accuracy
  correct_test = pred_test == y_test
  accuracy_test[k] = np.mean(correct_test.cpu().numpy())
      
  # compute training time
  stop = time.time()
  duration = stop - start
  time1[k] = duration
      
  # print results
  print('Epoch', k + 1, '/', niter, '\n', 
        '-', 'time:', time1[k], '-', 'sq_loss:', loss1[k], '-', 'tot_loss:', loss2[k], 
        '-', 'acc:', accuracy_train[k], '-', 'val_acc:', accuracy_test[k])

Train on 60000 samples, validate on 10000 samples
Epoch 1 / 10 
 - time: 1.0639472007751465 - sq_loss: 27050.24609375 - tot_loss: 1249918.446548462 - acc: 0.21726666666666666 - val_acc: 0.212
Epoch 2 / 10 
 - time: 1.0043509006500244 - sq_loss: 24011.025390625 - tot_loss: 1245972.4578552246 - acc: 0.15388333333333334 - val_acc: 0.1503
Epoch 3 / 10 
 - time: 1.0123765468597412 - sq_loss: 21430.466796875 - tot_loss: 1025423.5185546875 - acc: 0.08055 - val_acc: 0.0798
Epoch 4 / 10 
 - time: 1.0131340026855469 - sq_loss: 18921.751953125 - tot_loss: 880212.1356811523 - acc: 0.11061666666666667 - val_acc: 0.1118
Epoch 5 / 10 
 - time: 1.0169203281402588 - sq_loss: 17209.11328125 - tot_loss: 739629.3952636719 - acc: 0.22581666666666667 - val_acc: 0.2198
Epoch 6 / 10 
 - time: 1.0206010341644287 - sq_loss: 15114.0615234375 - tot_loss: 641552.8395996094 - acc: 0.1861 - val_acc: 0.1827
Epoch 7 / 10 
 - time: 1.0090179443359375 - sq_loss: 14096.0419921875 - tot_loss: 587621.8549804688 - acc: 0.02

In [51]:
W1 = W1*(3/2)
W2 = W2*(9/4)
W3 = W3*(3/2)
b1 = b1*(3/2)
b2 = b2*(3/2)

a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), W2, a1_test))
pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), W3, a2_test), dim=0)

U1 = torch.addmm(b1.repeat(1, N), W1, x_train) # equivalent to W1@x_train+b1.repeat(1,N)
V1 = nn.ReLU()(U1)
U2 = torch.addmm(b2.repeat(1, N), W2, V1)
V2 = nn.ReLU()(U2)
U3 = torch.addmm(b3.repeat(1, N), W3, V2)
V3 = U3
loss1 = gamma3/2*torch.pow(torch.dist(V3,y_train_one_hot,2),2).cpu().numpy()

correct_test = pred_test == y_test
accuracy_test = np.mean(correct_test.cpu().numpy())

In [52]:
## We plot the train losses
print(loss1)
#plt.xlabel('epochs')
#plt.ylabel('train losses')
#plt.plot(np.arange(0,loss1.shape[0]), loss1)
#plt.show()

416506.75


In [53]:
### We plot the test accuracy
print(accuracy_test)
#plt.xlabel('epochs')
#plt.ylabel('test accuracy')
#plt.plot(np.arange(0, accuracy_test.shape[0]), accuracy_test)
#plt.show()

0.6923
